In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import os
os.listdir('.')

['.config', 'sample_data']

In [9]:
df = pd.read_csv('raw_data.csv', skipinitialspace=True, sep=',', index_col=0)
df.head()

,год,время_года,климат,город,страна,способ_охлаждения,режим_при_смешанном_типе_охлаждения,способ_обогрева,возраст,пол,...,скорость_воздуха,рост,вес,занавески,вентилятор,окно,двери,отопление,среднемесячная_температура_на_улице,количество_рекламаций
0,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,22.0,NaN,...,0.26,NaN,NaN,1.0,1.0,NaN,NaN,NaN,28.9,0
1,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,40.0,NaN,...,0.12,NaN,NaN,0.0,0.0,NaN,NaN,NaN,84.0,0
2,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,42.0,NaN,...,0.08,NaN,NaN,0.0,0.0,NaN,NaN,NaN,32.8,0
3,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,27.0,NaN,...,27.16,NaN,NaN,1.0,1.0,NaN,NaN,NaN,28.9,1
4,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,46.0,NaN,...,63.83,NaN,NaN,1.0,1.0,NaN,NaN,NaN,32.8,0


In [ ]:
# обработка сырых данных
# df.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)
# df['климат'].replace('Cубтроп океанич', 'Cубтропический океанический', inplace=True)
# df['предпочтительное_изменение_температуры'].replace({'Холодн': 'Холоднее', 'Тепле': 'Теплее'}, inplace=True)
# df['режим_при_смешанном_типе_охлаждения'] = df['режим_при_смешанном_типе_охлаждения'].fillna('нет')
# df['способ_обогрева'] = df['способ_обогрева'].fillna('нет')
# df.to_csv('raw_data.csv')

In [ ]:
# заполняем возраст
gb = df.groupby('город')['возраст']
md = gb.median()
for city in md.index:
    if not md.isna()[city]:
        df['возраст'][df['город'] == city] = df['возраст'][df['город'] == city].fillna(md[city])
    else:
        df['возраст'][df['город'] == city] = df['возраст'][df['город'] == city].fillna(df['возраст'].median())

In [ ]:
# заполняем пол
gb = df.groupby('город')['пол']
glbrel = df['пол'].value_counts()['Мужской'] / df['пол'].value_counts()['Женский']
for key, it in gb:
    vc = it.value_counts()
    if vc.size and vc['Мужской'] and vc['Женский']:
        rel = vc['Мужской'] / vc['Женский']
    else:
        rel = glbrel
    ln = len(it)
    f = int(round(ln / (rel + 1)))
    m = ln - f
    df['пол'].fillna({it.index[i]: ('Мужской' if i < m else 'Женский') for i in range(ln)}, inplace=True)

In [ ]:
# заполняем рост
df["возраст"] = df['возраст'].astype(int)
df.loc[df['рост'] <= df['рост'].quantile(0.01), 'рост'] = np.nan
gb1 = df.groupby(['возраст'])['рост']
for key, item in gb1:
    key_mean = gb1.get_group(key).median()
    df["рост"] = df["рост"].fillna(key_mean)

In [ ]:
# заполняем вес
gb2 = df.groupby(['возраст'])['вес']
for key, item in gb2:
    key_mean = gb2.get_group(key).median()
    df["вес"] = df["вес"].fillna(key_mean)

In [ ]:
# заполняем температуру воздуха на улице
gbl = ['климат', 'время_года']
gb = df.groupby(gbl)['температура_воздуха_на_улице']
md = gb.median()
for key in md.index:
    condition = df['температура_воздуха_на_улице'] != -1
    for i in range(len(gbl)):
        condition = condition & (df[gbl[i]] == key[i])
    if not md.isna()[key]:
        df['температура_воздуха_на_улице'][condition] = df['температура_воздуха_на_улице'][condition].fillna(md[key])
    else:
        df['температура_воздуха_на_улице'][condition] = df['температура_воздуха_на_улице'][condition].fillna(df['температура_воздуха_на_улице'].median())

In [ ]:
# заполняем занавески
vc = df['занавески'].value_counts()
rel = vc[1] / vc[0]
z = int(round(len(df) / (rel + 1)))
o = ln - z
df['занавески'].fillna({i: (1 if i < o else 0) for i in df.index}, inplace=True)

In [10]:
# заполняем окно и двери
def zapolnenie(st2):
    gb = df.groupby(['климат','время_года'])[st2.name]
    dolya = 0
    for group_name, df_group in gb:
        if df_group.isna().sum() == 0:
            dolya_0 = df_group.value_counts()[0.0]/(len(df_group))
            dolya += dolya_0
    dolya = dolya/len(df_group)
    for group_name, df_group in gb:
        if df_group.isna().sum() != 0:
            kolvo_0 = int(round(dolya * len(df_group),0))
            df[st2.name] = df[st2.name].fillna({df_group.index[i]: (0.0 if i < kolvo_0 else 1.0) for i in range(len(df_group))})

zapolnenie(df['окно'])
zapolnenie(df['двери'])

# заполняем отопление
gb = df.groupby(['климат','время_года'])['отопление']
for group_name, df_group in gb:
    moda = df_group.mode()
    df['отопление'] = df['отопление'].fillna({df_group.index[i]: (moda) for i in range(len(df_group))})

In [ ]:
df.to_csv('corrected_data.csv')